<a href="https://colab.research.google.com/github/smcdonou02/McDonut/blob/main/TournamentDES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dependencies / Imports

In [9]:
import simpy
import numpy as np
import pandas as pd
import math
import random
from collections import defaultdict

User Config

In [10]:
CONFIG_FILE = "/content/drive/MyDrive/2024-2025 /_SSY611_Team_Project/Python Simulation Script/Simulation Script/ADCC_2025_US_Open - Orlando_FL_Config_From_Raw.xlsx"

SCENARIOS = {
    "A": {"no_show_dq": 0.07, "submission": 0.38, "points": 0.55},
    "B": {"no_show_dq": 0.05, "submission": 0.55, "points": 0.40},
    "C": {"no_show_dq": 0.15, "submission": 0.35, "points": 0.50},
    "D": {"no_show_dq": 0.05, "submission": 0.25, "points": 0.70},
}

SELECTED_SCENARIO = "B"

Constants/ Key Parameters

In [11]:
NUM_MATS = 4
CHECKIN_STAFF = 10
ARRIVAL_START = -30.0
ARRIVAL_END = 0.0
BRACKET_START = 0.0
CHECKIN_MIN = 0.5
CHECKIN_MAX = 1.5
BUFFER_MIN = 1.0
BUFFER_MAX = 3.0
RANDOM_SEED = 42

In [12]:
#import data file, clean up any bad data
def load_tournament_divisions(path):
  df = pd.read_excel(path)
  df["Num_Competitors"] = pd.to_numeric(df["Num_Competitors"], errors="coerce").fillna(0).astype(int)
  df["Match_Time_Min"] = pd.to_numeric(df["Match_Time_Min"], errors="coerce").fillna(5.0).astype(float)
  return df

def compute_round_info(n):
        # Single elimination rounds
        rounds = []
        competitors = n
        while competitors > 1:
            matches = competitors // 2
            byes = competitors % 2
            rounds.append({"matches": matches, "byes": byes, "players_in": competitors})
            competitors = matches + byes
        return rounds

In [13]:
class TournamentDES:
    def __init__(self, env, div_row, scen_probs, mats_resource, checkin_resource,
                 buffer_min, buffer_max, arrival_start, arrival_end, bracket_start,
                 checkin_min, checkin_max, rng):
        self.env = env
        self.div = div_row
        self.probs = scen_probs
        self.mats = mats_resource
        self.checkin = checkin_resource
        self.buffer_min = buffer_min
        self.buffer_max = buffer_max
        self.arrival_start = arrival_start
        self.arrival_end = arrival_end
        self.bracket_start = bracket_start
        self.checkin_min = checkin_min
        self.checkin_max = checkin_max
        self.rng = rng

        self.n = int(div_row["Num_Competitors"])
        self.match_time = float(div_row["Match_Time_Min"])
        self.code = div_row["Division_Code"]

        self.round_info = compute_round_info(self.n)
        self.num_rounds = len(self.round_info)
        self.round_stores = [simpy.Store(env) for _ in range(self.num_rounds + 1)]

        self.match_logs = []
        self.wait_times_round = defaultdict(list)
        self.round_start_times = [None] * (self.num_rounds + 1)
        self.round_end_times = [None] * (self.num_rounds + 1)
        self.enter_time = {}

        self.wait_times_checkin = []
        self.queue_lengths_checkin = []
        self.mat_occupancy_start = []
        self.mat_occupancy_end = []



    def start(self):
        for cid in range(1, self.n + 1):
            self.env.process(self.competitor_process(cid))
        self.env.process(self.bracket_orchestrator())
        return self._generator()

    def _generator(self):
        while True:
            yield self.env.timeout(1)

    def competitor_process(self, competitor_id):
      t_rel = self.rng.uniform(self.arrival_start, self.arrival_end)
      arrival_time = max(0.0, self.bracket_start + t_rel)
      yield self.env.timeout(max(0, arrival_time - self.env.now))

      arrive = self.env.now

      with self.checkin.request() as req:
        qstart = self.env.now
        yield req
        wait_ci = self.env.now - qstart  # time spent waiting
        self.wait_times_checkin.append(wait_ci)  # store for stats
        self.queue_lengths_checkin.append(len(self.checkin.queue))  # track queue length

        svc = self.rng.uniform(self.checkin_min, self.checkin_max)
        yield self.env.timeout(svc)

      self.enter_time[(competitor_id, 0)] = self.env.now
      yield self.round_stores[0].put(competitor_id)

    def bracket_orchestrator(self):
        # wait until bracket start
        if self.env.now < self.bracket_start:
            yield self.env.timeout(self.bracket_start - self.env.now)

        # schedule all rounds
        for r in range(self.num_rounds):
            self.round_start_times[r] = self.env.now
            yield self.env.process(self.round_scheduler(r))
            self.round_end_times[r] = self.env.now

        # Final winner collection
        final_round = self.num_rounds
        needed = self.round_info[-1]["players_in"] if self.round_info else 1
        for _ in range(needed):
            item = yield self.round_stores[final_round].get()
            yield self.round_stores[final_round].put(item)  # keep in store

    def round_scheduler(self, r):
        round_info = self.round_info[r]
        matches = round_info["matches"]
        byes = round_info["byes"]

        match_procs = []

        # schedule matches
        for m in range(matches):
            compA = yield self.round_stores[r].get()
            compB = yield self.round_stores[r].get()

            timeA = self.enter_time.pop((compA, r), self.env.now)
            timeB = self.enter_time.pop((compB, r), self.env.now)
            self.wait_times_round[r].append(self.env.now - timeA)
            self.wait_times_round[r].append(self.env.now - timeB)

            match_procs.append(self.env.process(self.run_match(r, compA, compB)))

        # schedule lone competitor if bye
        if byes == 1:
            lone = yield self.round_stores[r].get()
            timeL = self.enter_time.pop((lone, r), self.env.now)
            self.wait_times_round[r].append(self.env.now - timeL)
            self.enter_time[(lone, r + 1)] = self.env.now
            yield self.round_stores[r + 1].put(lone)

        # wait for all matches in this round to finish
        if match_procs:
            yield simpy.events.AllOf(self.env, match_procs)

    def run_match(self, round_index, compA, compB):
        with self.mats.request() as req:
            yield req
            match_start = self.env.now

            self.mat_occupancy_start.append((match_start, self.mats.count))

            keys = ["no_show_dq", "submission", "points"]
            probs = [self.probs[k] for k in keys]
            outcome = self.rng.choice(keys, p=probs)

            if outcome == "no_show_dq":
                completed_time = 0.0
            elif outcome == "submission":
                completed_time = float(self.rng.uniform(0.1, max(0.2, self.match_time - 0.1)))
            else:
                completed_time = float(self.match_time)

            if completed_time > 0.0:
                yield self.env.timeout(completed_time)

            # buffer time
            if completed_time == 0.0:
                buff = float(self.rng.uniform(max(0.1, self.buffer_min / 2.0), max(0.5, self.buffer_min)))
            else:
                buff = float(self.rng.uniform(self.buffer_min, self.buffer_max))
            yield self.env.timeout(buff)

            match_end = self.env.now
            self.mat_occupancy_end.append((match_end, self.mats.count))

            winner = compA if self.rng.random() < 0.5 else compB

            log = {
              "Division": self.code,
              "Round": round_index + 1,
              "Match_Pair": (compA, compB),
              "Winner": winner,
              "Outcome": outcome,
              "Match_Start": match_start,
              "Match_End": match_end,
              "Completed_Time": completed_time,
              "Buffer_Time": buff,
              "Total_Mat_Occupancy": completed_time,
            }
            self.match_logs.append(log)

            if (round_index + 1) <= self.num_rounds:
                self.enter_time[(winner, round_index + 1)] = self.env.now
                yield self.round_stores[round_index + 1].put(winner)


In [14]:
def run_simulation(div_df, scenario="A", num_mats=NUM_MATS, checkin_staff=CHECKIN_STAFF,
                   arrival_start=ARRIVAL_START, arrival_end=ARRIVAL_END, bracket_start=BRACKET_START,
                   checkin_min=CHECKIN_MIN, checkin_max=CHECKIN_MAX,
                   buffer_min=BUFFER_MIN, buffer_max=BUFFER_MAX, seed=RANDOM_SEED):
    rng = np.random.default_rng(seed)

    env = simpy.Environment()
    mats = simpy.Resource(env, capacity=num_mats)
    checkin = simpy.Resource(env, capacity=checkin_staff)

    all_match_logs = []
    division_summaries = []

    des_objects = []
    for _, row in div_df.iterrows():
      if int(row["Num_Competitors"]) < 2:
        division_summaries.append({
        "Division": row["Division_Code"],
        "Num_Competitors": row["Num_Competitors"],
        "Num_Matches": 0,
        "Division_Complete_Time": 0.0
        })
        continue

    tdes = TournamentDES(env, row, SCENARIOS[scenario], mats, checkin,
                             buffer_min, buffer_max, arrival_start, arrival_end,
                             bracket_start, checkin_min, checkin_max, rng)
    des_objects.append(tdes)
    tdes.start()

    MAX_SIM_TIME = 24 * 60
    finished_divs = set()

    dt = 1.0
    while env.now < MAX_SIM_TIME:
        env.step()
        for tdes in des_objects:
          if tdes.code in finished_divs:
            continue
          final_store = tdes.round_stores[tdes.num_rounds]
          if len(final_store.items) >= 1:
            all_match_logs.extend(tdes.match_logs)
            last_match_end = max([m["Match_End"] for m in tdes.match_logs]) if tdes.match_logs else env.now
            division_summaries.append({
                "Division": tdes.code,
                "Num_Competitors": tdes.n,
                "Num_Matches": sum([ri["matches"] for ri in tdes.round_info]),
                "Division_Complete_Time": last_match_end
            })
            finished_divs.add(tdes.code)
        if len(finished_divs) == len(des_objects):
          break
    for tdes in des_objects:
      if tdes.code not in finished_divs:
        all_match_logs.extend(tdes.match_logs)
        last_match_end = max([m["Match_End"] for m in tdes.match_logs])
        division_summaries.append({
            "Num_Competitors": tdes.n,
            "Num_Matches": sum([ri["matches"] for ri in tdes.round_info]),
            "Division_Complete_Time": last_match_end
        })

    matches_df = pd.DataFrame(all_match_logs)
    summary_df = pd.DataFrame(division_summaries)

    all_checkin_waits = []
    all_checkin_queues = []
    for tdes in des_objects:
      all_checkin_waits.extend(tdes.wait_times_checkin)
      all_checkin_queues.extend(tdes.queue_lengths_checkin)

    avg_checkin_wait = np.mean(all_checkin_waits) if all_checkin_waits else 0
    max_checkin_queue = max(all_checkin_queues) if all_checkin_queues else 0
    round_durations = [end - start for start, end in zip(tdes.round_start_times, tdes.round_end_times) if end]

    used_mat_time = sum(
      end-start for tdes in des_objects
      for (start,_), (end,_) in zip(tdes.mat_occupancy_start, tdes.mat_occupancy_end))
    total_possible = num_mats * env.now
    utilization = used_mat_time / total_possible if total_possible > 0 else 0
    tournament_finish = summary_df["Division_Complete_Time"].max() if not summary_df.empty else 0.0

    return {
      "env_time": env.now,
      "matches_df": matches_df,
      "summary_df": summary_df,
      "tournament_finish_min": tournament_finish,
      "avg_checkin_wait": avg_checkin_wait,
      "max_checkin_queue": max_checkin_queue,
      "round_durations": round_durations,
      "mat_utilization": utilization
}




In [15]:
if __name__ == "__main__":
  np.random.seed(RANDOM_SEED)
  divs = load_tournament_divisions(CONFIG_FILE)
  print("\n--- Raw Tournament Data ---")
  print(divs.head())          # show first 5 rows
  print(divs.info())          # check column types and non-null counts
  print(divs.describe())      # summary statistics for numeric columns

  # Optional: see round info for each division
  for idx, row in divs.iterrows():
    n = row["Num_Competitors"]
    rounds = compute_round_info(n)
    print(f"\nDivision {row['Division_Code']} ({n} competitors):")
    for r_idx, r in enumerate(rounds):
        print(f"  Round {r_idx+1}: {r['matches']} matches, {r['byes']} byes")

  result = run_simulation(divs, scenario=SELECTED_SCENARIO, num_mats=NUM_MATS,
                            checkin_staff=CHECKIN_STAFF,
                            arrival_start=ARRIVAL_START, arrival_end=ARRIVAL_END,
                            bracket_start=BRACKET_START,
                            checkin_min=CHECKIN_MIN, checkin_max=CHECKIN_MAX,
                            buffer_min=BUFFER_MIN, buffer_max=BUFFER_MAX,
                            seed=RANDOM_SEED)



--- Raw Tournament Data ---
        Division_Code Age_Label    Belt_Level Weight_Class  Num_Competitors  \
0  Intermediate_550kg  -55,0 kg  Intermediate          NaN                5   
1      Advanced_280kg  -28,0 kg      Advanced          NaN                4   
2  Intermediate_280kg  -28,0 kg  Intermediate          NaN                4   
3  Intermediate_400kg  -40,0 kg  Intermediate          NaN                5   
4  Intermediate_200kg  -20,0 kg  Intermediate          NaN                5   

   Match_Time_Min   Group_Description Start_Time     Mat  
0             5.0  Boys [13-14 years]   08:05:00  Mat 10  
1             5.0   Girls [7-8 years]   08:05:00   Mat 5  
2             5.0   Girls [7-8 years]   08:06:00   Mat 4  
3             5.0  Boys [11-12 years]   08:07:00   Mat 8  
4             5.0  Boys [6 and under]   08:07:00   Mat 3  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 9 columns):
 #   Column             Non-Null Count

In [16]:
print("\n--- Sim Summary ---")
print(f"Sim clock: {result['env_time']:.2f} minutes")
print(f"Tournament finish (max division): {result['tournament_finish_min']:.2f} minutes")
print(f"Average check-in wait: {result['avg_checkin_wait']:.2f} minutes")
print(f"Max check-in queue length: {result['max_checkin_queue']}")
print(f"Mat utilization: {result['mat_utilization']*100:.2f}%")
print("\nDivision summary:")
print(result["summary_df"])
print("\nSample match logs:")
print(result["matches_df"])


--- Sim Summary ---
Sim clock: 19.08 minutes
Tournament finish (max division): 19.08 minutes
Average check-in wait: 0.00 minutes
Max check-in queue length: 0
Mat utilization: 53.12%

Division summary:
                 Division  Num_Competitors  Num_Matches  \
0  Adult15_Beginner_760kg                8            7   

   Division_Complete_Time  
0               19.078303  

Sample match logs:
                 Division  Round Match_Pair  Winner     Outcome  Match_Start  \
0  Adult15_Beginner_760kg      1     (1, 8)       8  submission     0.727239   
1  Adult15_Beginner_760kg      1     (2, 5)       2      points     1.143865   
2  Adult15_Beginner_760kg      1     (6, 4)       6  submission     1.426765   
3  Adult15_Beginner_760kg      1     (3, 7)       7      points     0.943414   
4  Adult15_Beginner_760kg      2     (8, 2)       8      points     8.500181   
5  Adult15_Beginner_760kg      2     (6, 7)       6      points     8.500181   
6  Adult15_Beginner_760kg      3     (8, 6)